In [1]:
import tensorflow as tf
import numpy as np
import utils
import sys

import keras
from keras.layers import Input, LSTM, Reshape, Dense, Lambda
from keras.optimizers import Adam
from keras.models import Model

Using TensorFlow backend.


In [2]:
def lstmModel1Layer(Tx, n_h , n_values):
    
    X_inp = Input(shape=(Tx, n_values))
    
    a0 = Input(shape=(n_h,), name='a0')
    c0 = Input(shape=(n_h,), name='c0')
    a = a0
    c = c0
    
    outputs = []
    
    for t in range(Tx):
        
        x = Lambda(lambda x: X_inp[:, t,:])(X_inp)
        x = reshapor(x)
        a, _, c = LSTM_cell_layer1(x, initial_state=[a, c])
        out = densor(a)
        outputs.append(out)
        
    model = Model([X_inp, a0, c0], outputs)
    
    return model

In [3]:
max_model = None
max_score = -sys.maxsize - 1

n_h = 16
folds = 5

NUM_EPOCHS = 20
BATCH_SIZE = 2

cvscores = []

opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

for i in range(1, folds+1):

    # Input shape (M, T, P)
    X_train, Y_train, X_test, Y_test = utils.getAnnotation(cross_validation=i)
    
    M, Tx, n_values = X_train.shape
    Ty = Tx
    
    reshapor = Reshape((1, n_values))
    LSTM_cell_layer1 = LSTM(n_h, return_state = True)
    densor = Dense(1, activation='sigmoid')
    
    model = lstmModel1Layer(Tx, n_h, n_values)
    
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    a0 = np.zeros((BATCH_SIZE, n_h))
    c0 = np.zeros((BATCH_SIZE, n_h))
 
    examples = int(M/BATCH_SIZE)
    st = 0
    end = st + BATCH_SIZE
    for b in range(examples):
        model.fit([X_train[st:end, :, :], a0, c0], list(Y_train[:, st:end, :]), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)
        st += 2
        end = st + BATCH_SIZE
    
    st = 0
    end = st + BATCH_SIZE
    
    examples = int(X_test.shape[0]/BATCH_SIZE)
    
    total = []
    for b in range(examples):
        model.evaluate([X_test[st:end, :, :], a0, c0], list(Y_test[:, st:end, :]), batch_size=BATCH_SIZE)
        
    if i == 1:
        max_model = model
        model.summary()
        break

Epoch 1/20
2/2 [==============================] - 3s 2s/step - loss: 5.7040 - dense_1_loss_1: 0.4966 - dense_1_loss_2: 0.4141 - dense_1_loss_3: 0.4015 - dense_1_loss_4: 0.6457 - dense_1_loss_5: 0.6568 - dense_1_loss_6: 0.2862 - dense_1_loss_7: 0.2873 - dense_1_loss_8: 0.2875 - dense_1_loss_9: 0.2875 - dense_1_loss_10: 0.2875 - dense_1_loss_11: 0.3278 - dense_1_loss_12: 0.2933 - dense_1_loss_13: 0.3900 - dense_1_loss_14: 0.3285 - dense_1_loss_15: 0.3137 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 1.0000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 0.5000 - dense_1_acc_5: 0.5000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 2/20
2/2 [==============================] - 0s 19ms/step - loss: 5.1945 - dense_1_loss_1: 0.4417 - dense_1_loss_2: 0.3920 - dense_1_loss_3: 0.3511 - dense_1_loss_4: 0.6467

Epoch 11/20
2/2 [==============================] - 0s 20ms/step - loss: 4.2643 - dense_1_loss_1: 0.2574 - dense_1_loss_2: 0.1985 - dense_1_loss_3: 0.1605 - dense_1_loss_4: 1.0491 - dense_1_loss_5: 0.9962 - dense_1_loss_6: 0.1517 - dense_1_loss_7: 0.1517 - dense_1_loss_8: 0.1517 - dense_1_loss_9: 0.1517 - dense_1_loss_10: 0.1517 - dense_1_loss_11: 0.1595 - dense_1_loss_12: 0.1595 - dense_1_loss_13: 0.1973 - dense_1_loss_14: 0.1673 - dense_1_loss_15: 0.1606 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 1.0000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 0.5000 - dense_1_acc_5: 0.5000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 12/20
2/2 [==============================] - 0s 19ms/step - loss: 4.2575 - dense_1_loss_1: 0.2470 - dense_1_loss_2: 0.1897 - dense_1_loss_3: 0.1526 - dense_1_loss_4: 1.

Epoch 1/20
2/2 [==============================] - 0s 18ms/step - loss: 3.5166 - dense_1_loss_1: 0.1922 - dense_1_loss_2: 0.1221 - dense_1_loss_3: 0.1013 - dense_1_loss_4: 0.1048 - dense_1_loss_5: 0.1220 - dense_1_loss_6: 0.1220 - dense_1_loss_7: 1.0083 - dense_1_loss_8: 1.0083 - dense_1_loss_9: 0.0917 - dense_1_loss_10: 0.1022 - dense_1_loss_11: 0.0892 - dense_1_loss_12: 0.1251 - dense_1_loss_13: 0.1200 - dense_1_loss_14: 0.1155 - dense_1_loss_15: 0.0919 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 1.0000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 0.5000 - dense_1_acc_8: 0.5000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 2/20
2/2 [==============================] - 0s 18ms/step - loss: 3.4516 - dense_1_loss_1: 0.1869 - dense_1_loss_2: 0.1252 - dense_1_loss_3: 0.1022 - dense_1_loss_4: 0.09

Epoch 11/20
2/2 [==============================] - 0s 18ms/step - loss: 3.1819 - dense_1_loss_1: 0.1518 - dense_1_loss_2: 0.0864 - dense_1_loss_3: 0.0731 - dense_1_loss_4: 0.0715 - dense_1_loss_5: 0.0713 - dense_1_loss_6: 0.0712 - dense_1_loss_7: 1.0998 - dense_1_loss_8: 1.0998 - dense_1_loss_9: 0.0616 - dense_1_loss_10: 0.0717 - dense_1_loss_11: 0.0579 - dense_1_loss_12: 0.0611 - dense_1_loss_13: 0.0641 - dense_1_loss_14: 0.0788 - dense_1_loss_15: 0.0619 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 1.0000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 0.5000 - dense_1_acc_8: 0.5000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 12/20
2/2 [==============================] - 0s 18ms/step - loss: 3.1680 - dense_1_loss_1: 0.1491 - dense_1_loss_2: 0.0845 - dense_1_loss_3: 0.0716 - dense_1_loss_4: 0.

Epoch 1/20
2/2 [==============================] - 0s 18ms/step - loss: 1.6215 - dense_1_loss_1: 0.1281 - dense_1_loss_2: 0.0870 - dense_1_loss_3: 0.1104 - dense_1_loss_4: 0.1236 - dense_1_loss_5: 0.1404 - dense_1_loss_6: 0.1760 - dense_1_loss_7: 0.1270 - dense_1_loss_8: 0.1757 - dense_1_loss_9: 0.0983 - dense_1_loss_10: 0.0983 - dense_1_loss_11: 0.0980 - dense_1_loss_12: 0.0664 - dense_1_loss_13: 0.0629 - dense_1_loss_14: 0.0672 - dense_1_loss_15: 0.0623 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 1.0000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 2/20
2/2 [==============================] - 0s 19ms/step - loss: 1.5712 - dense_1_loss_1: 0.1028 - dense_1_loss_2: 0.0854 - dense_1_loss_3: 0.1084 - dense_1_loss_4: 0.13

...

Epoch 1/20
2/2 [==============================] - 0s 18ms/step - loss: 2.4649 - dense_1_loss_1: 0.0252 - dense_1_loss_2: 2.2945 - dense_1_loss_3: 0.0132 - dense_1_loss_4: 0.0120 - dense_1_loss_5: 0.0121 - dense_1_loss_6: 0.0122 - dense_1_loss_7: 0.0121 - dense_1_loss_8: 0.0121 - dense_1_loss_9: 0.0117 - dense_1_loss_10: 0.0116 - dense_1_loss_11: 0.0114 - dense_1_loss_12: 0.0094 - dense_1_loss_13: 0.0091 - dense_1_loss_14: 0.0091 - dense_1_loss_15: 0.0091 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 0.5000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 2/20
2/2 [==============================] - 0s 18ms/step - loss: 2.4666 - dense_1_loss_1: 0.0251 - dense_1_loss_2: 2.2974 - dense_1_loss_3: 0.0132 - dense_1_loss_4: 0.01

Epoch 11/20
2/2 [==============================] - 0s 18ms/step - loss: 2.4509 - dense_1_loss_1: 0.0260 - dense_1_loss_2: 2.2706 - dense_1_loss_3: 0.0137 - dense_1_loss_4: 0.0128 - dense_1_loss_5: 0.0126 - dense_1_loss_6: 0.0126 - dense_1_loss_7: 0.0126 - dense_1_loss_8: 0.0126 - dense_1_loss_9: 0.0126 - dense_1_loss_10: 0.0126 - dense_1_loss_11: 0.0119 - dense_1_loss_12: 0.0099 - dense_1_loss_13: 0.0102 - dense_1_loss_14: 0.0102 - dense_1_loss_15: 0.0099 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 0.5000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 12/20
2/2 [==============================] - 0s 18ms/step - loss: 2.4463 - dense_1_loss_1: 0.0263 - dense_1_loss_2: 2.2639 - dense_1_loss_3: 0.0139 - dense_1_loss_4: 0.

Epoch 1/20
2/2 [==============================] - 0s 18ms/step - loss: 2.4460 - dense_1_loss_1: 0.0340 - dense_1_loss_2: 0.0165 - dense_1_loss_3: 0.0128 - dense_1_loss_4: 0.0109 - dense_1_loss_5: 2.2450 - dense_1_loss_6: 0.0177 - dense_1_loss_7: 0.0177 - dense_1_loss_8: 0.0117 - dense_1_loss_9: 0.0115 - dense_1_loss_10: 0.0115 - dense_1_loss_11: 0.0112 - dense_1_loss_12: 0.0115 - dense_1_loss_13: 0.0112 - dense_1_loss_14: 0.0115 - dense_1_loss_15: 0.0112 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 1.0000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 0.5000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 2/20
2/2 [==============================] - 0s 19ms/step - loss: 2.4403 - dense_1_loss_1: 0.0344 - dense_1_loss_2: 0.0170 - dense_1_loss_3: 0.0130 - dense_1_loss_4: 0.01

Epoch 11/20
2/2 [==============================] - 0s 18ms/step - loss: 2.3903 - dense_1_loss_1: 0.0384 - dense_1_loss_2: 0.0195 - dense_1_loss_3: 0.0153 - dense_1_loss_4: 0.0130 - dense_1_loss_5: 2.1515 - dense_1_loss_6: 0.0211 - dense_1_loss_7: 0.0211 - dense_1_loss_8: 0.0140 - dense_1_loss_9: 0.0139 - dense_1_loss_10: 0.0139 - dense_1_loss_11: 0.0136 - dense_1_loss_12: 0.0139 - dense_1_loss_13: 0.0138 - dense_1_loss_14: 0.0139 - dense_1_loss_15: 0.0135 - dense_1_acc_1: 1.0000 - dense_1_acc_2: 1.0000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 0.5000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000
Epoch 12/20
2/2 [==============================] - 0s 18ms/step - loss: 2.3852 - dense_1_loss_1: 0.0389 - dense_1_loss_2: 0.0198 - dense_1_loss_3: 0.0156 - dense_1_loss_4: 0.

2/2 [==============================] - 0s 5ms/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15, 6)        0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 6)            0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 6)         0           lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
                                                          

In [4]:
X_predict, f_names = utils.getPredictingData()
print(X_predict.shape)
out = max_model.predict([X_predict, a0, c0], batch_size = BATCH_SIZE)
print(out)
print(f_names)

(1, 15, 6)
[array([[ 0.04219071]], dtype=float32), array([[ 0.02524883]], dtype=float32), array([[ 0.02332149]], dtype=float32), array([[ 0.02306747]], dtype=float32), array([[ 0.02303322]], dtype=float32), array([[ 0.02302859]], dtype=float32), array([[ 0.02302796]], dtype=float32), array([[ 0.01780834]], dtype=float32), array([[ 0.01662869]], dtype=float32), array([[ 0.0164548]], dtype=float32), array([[ 0.0164548]], dtype=float32), array([[ 0.0164548]], dtype=float32), array([[ 0.0164548]], dtype=float32), array([[ 0.0164548]], dtype=float32), array([[ 0.0164548]], dtype=float32)]
['data9-frame50.jpg', 'data9-frame56.jpg', 'data9-frame59.jpg', 'data9-frame60.jpg', 'data9-frame61.jpg', 'data9-frame70.jpg', 'data9-frame74.jpg', 'data9-frame79.jpg', 'data9-frame85.jpg', 'data9-frame100.jpg', 'data9-frame107.jpg', 'data9-frame109.jpg', 'data9-frame110.jpg', 'data9-frame114.jpg', 'data9-frame115.jpg']
